In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = "10019032" # 店舗ID
SPREADSHEET_KEY = '1NlYAskEPicsmVQCFukbouHeXl6-aAun6JuHt-H-Ll64' # スプレッドシートのID
pstart = 1 #1 パチンコの1台目
sstart = 433 #433 スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+456@gmail.com"
user_pw = "paruparu22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p2row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p2table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "20":
                    p2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "20":
                        p2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "549":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "549":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.2RAW'!F" + str(int(p2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p2table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/18', '025489', '1', '13333.0', '-1805.0', '5', '1']
['08/18', '025489', '2', '10256.0', '-902.0', '4', '2']
['08/18', '025489', '3', '34359.0', '-19258.0', '4', '3']
['08/18', '025489', '4', '5128.0', '-902.0', '2', '1']
['08/18', '025489', '5', '35385.0', '4816.0', '18', '4']
['08/18', '025489', '6', '35641.0', '14445.0', '24', '7']
['08/18', '025489', '7', '24103.0', '-13240.0', '3', '3']
['08/18', '025489', '8', '17692.0', '3913.0', '10', '4']
['08/18', '025489', '11', '25897.0', '-8726.0', '6', '5']
['08/18', '025489', '12', '9231.0', '-3911.0', '2', '1']
['08/18', '025489', '13', '22821.0', '10834.0', '16', '4']
['08/18', '025489', '14', '23333.0', '-12638.0', '3', '2']
['08/18', '025489', '15', '11282.0', '-3309.0', '3', '2']
['08/18', '025489', '16', '23590.0', '-2707.0', '8', '4']
['08/18', '025489', '17', '13333.0', '4214.0', '8', '3']
['08/18', '025489', '18', '30000.0', '-6319.0', '9', '5']
['08/18', '025895', '19', '27949.0', '17154.0', '27', '6']
['08/18', '025895', '

['08/18', '025864', '164', '4359.0', '-3911.0', '0', '0']
['08/18', '025775', '165', '0', '0', '0', '0']
['08/18', '025645', '166', '3590.0', '-3008.0', '0', '0']
['08/18', '025786', '167', '10769.0', '-1805.0', '8', '1']
['08/18', '025850', '168', '0', '0', '0', '0']
['08/18', '025761', '169', '2821.0', '-2406.0', '1', '1']
['08/18', '025839', '170', '2821.0', '-2106.0', '1', '1']
['08/18', '025590', '173', '27692.0', '-1805.0', '31', '31']
['08/18', '025599', '174', '4103.0', '-3309.0', '1', '1']
['08/18', '025775', '175', '2821.0', '-2406.0', '0', '0']
['08/18', '025837', '176', '4359.0', '-3610.0', '0', '0']
['08/18', '025306', '177', '769.0', '-1203.0', '0', '0']
['08/18', '025850', '178', '0', '0', '0', '0']
['08/18', '025701', '179', '3846.0', '-3309.0', '0', '0']
['08/18', '025550', '180', '2308.0', '-2106.0', '0', '0']
['08/18', '025779', '181', '4872.0', '-1203.0', '4', '2']
['08/18', '025779', '182', '30513.0', '-15045.0', '12', '4']
['08/18', '025779', '183', '0', '0', '0',

['08/18', '025364', '315', '23333.0', '-5717.0', '21', '9']
['08/18', '025636', '316', '12821.0', '-3911.0', '5', '1']
['08/18', '025765', '317', '21795.0', '-6619.0', '17', '8']
['08/18', '025655', '319', '16923.0', '-3610.0', '26', '6']
['08/18', '025636', '320', '4103.0', '-902.0', '3', '1']
['08/18', '025364', '321', '20769.0', '-4513.0', '18', '10']
['08/18', '025489', '322', '32564.0', '3913.0', '16', '8']
['08/18', '025553', '323', '43590.0', '1806.0', '37', '11']
['08/18', '025521', '324', '26667.0', '1806.0', '31', '9']
['08/18', '025825', '325', '16667.0', '6019.0', '14', '3']
['08/18', '025868', '326', '25128.0', '-5717.0', '28', '10']
['08/18', '025613', '327', '18205.0', '-13240.0', '4', '3']
['08/18', '025722', '328', '17436.0', '9029.0', '45', '6']
['08/18', '025613', '329', '18718.0', '1205.0', '17', '5']
['08/18', '025697', '330', '10513.0', '-6619.0', '7', '6']
['08/18', '025807', '331', '33590.0', '-11434.0', '24', '8']
['08/18', '025847', '332', '17692.0', '1506.0',

['08/18', '120117', '457', '2026', '-1435.0', '0', '4', '23']
['08/18', '120076', '458', '7326', '-370.0', '0', '0', '33']
['08/18', '120121', '459', '7522', '-417.0', '0', '16', '102']
['08/18', '119960', '461', '1373', '-509.0', '11', '7', '--']
['08/18', '120070', '462', '2333', '-1944.0', '0', '7', '9']
['08/18', '119990', '463', '3183', '1713.0', '0', '6', '35']
['08/18', '120034', '464', '5031', '1667.0', '0', '5', '50']
['08/18', '119984', '465', '7001', '1944.0', '25', '35', '--']
['08/18', '119993', '466', '1746', '-93.0', '0', '7', '12']
['08/18', '119897', '467', '4461', '-93.0', '31', '0', '29']
['08/18', '120048', '468', '2521', '93.0', '0', '4', '36']
['08/18', '120045', '469', '2422', '-1250.0', '0', '8', '33']
['08/18', '120103', '471', '2395', '2037.0', '31', '2', '4']
['08/18', '120103', '472', '698', '-93.0', '4', '0', '2']
['08/18', '120103', '473', '2056', '602.0', '23', '0', '7']
['08/18', '120128', '474', '4003', '-93.0', '0', '0', '13']
['08/18', '120128', '475'

['08/18', '120010', '596', '5305', '-1574.0', '13', '15', '--']
['08/18', '120010', '597', '1799', '-1111.0', '4', '0', '--']
['08/18', '120010', '598', '6672', '1019.0', '31', '17', '--']
['08/18', '120010', '599', '3149', '-509.0', '10', '6', '--']
['08/18', '120010', '600', '5286', '556.0', '21', '13', '--']
['08/18', '120010', '601', '7403', '1065.0', '32', '19', '--']
['08/18', '120141', '602', '2902', '-1204.0', '7', '8', '--']
['08/18', '120141', '603', '5086', '-1296.0', '18', '9', '--']
['08/18', '120141', '604', '4684', '-509.0', '16', '14', '--']
['08/18', '120141', '605', '6086', '-93.0', '23', '21', '--']
['08/18', '120141', '606', '3225', '139.0', '14', '7', '--']
['08/18', '120141', '607', '7811', '1250.0', '33', '30', '--']
['08/18', '119962', '609', '1188', '231.0', '6', '0', '--']
['08/18', '119962', '610', '2789', '185.0', '9', '9', '--']
['08/18', '119940', '611', '1035', '-1296.0', '2', '0', '--']
['08/18', '119940', '612', '1402', '-324.0', '8', '0', '5']
['08/18'

{'spreadsheetId': '1NlYAskEPicsmVQCFukbouHeXl6-aAun6JuHt-H-Ll64',
 'updates': {'spreadsheetId': '1NlYAskEPicsmVQCFukbouHeXl6-aAun6JuHt-H-Ll64',
  'updatedRange': 'S5RAW!F5510:M5543',
  'updatedRows': 34,
  'updatedColumns': 8,
  'updatedCells': 272}}